In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

In [2]:
test_data = np.load('/home/hrai/Datasets/HAAI/3DHP/poseaug/test_3dhp.npz', allow_pickle=True)

In [3]:
def add_nose_keypoint(pose2d, pose3d):
    assert pose2d.shape[1] == 16, "pose2d shape should be (N, 16, 2)"
    assert pose3d.shape[1] == 16, "pose3d shape should be (N, 16, 3)"
    new_pose2d = np.zeros((pose2d.shape[0], 17, 2))
    new_pose3d = np.zeros((pose3d.shape[0], 17, 3))
    
    new_pose2d[:, 9, :]   = (pose2d[:, 8, :] + pose2d[:, 9, :])/2
    new_pose2d[:, 0:9, :] =  pose2d[:, 0:9, :] # 0-8
    new_pose2d[:, 10:, :] =  pose2d[:, 9:, :]
    new_pose3d[:, 9, :]   = (pose3d[:, 8, :] + pose3d[:, 9, :])/2
    new_pose3d[:, 0:9, :] =  pose3d[:, 0:9, :] # 0-8
    new_pose3d[:, 10:, :] =  pose3d[:, 9:, :]
    
    return new_pose2d, new_pose3d

In [24]:
joint_2ds = {'train': [], 'test': []}
confidences = {'train': [], 'test': []}
camera_names = {'train': [], 'test': []}
actions = {'train': [], 'test': []}
sources = {'train': [], 'test': []}
frames = {'train': [], 'test': []}
cam_3ds = {'train': [], 'test': []}

# target 
only_test = True
subject = '3dhp'
data_type = 'test' 

print(test_data['pose2d'].shape, test_data['pose3d'].shape)
# 16 joints -> 17 joints # nose -> 9
new_pose2d, new_pose3d = add_nose_keypoint(test_data['pose2d'], test_data['pose3d'])

joint_2ds[data_type] += list(new_pose2d) # 
confidences[data_type] += list(np.ones_like(new_pose2d[:, :, 0:1]))
actions[data_type] += list([subject])*len(new_pose2d)
sources[data_type] += list([subject])*len(new_pose2d)
camera_names[data_type] += list([subject])*len(new_pose2d)
frames[data_type] += [i for i in range(len(new_pose2d))]
cam_3ds[data_type] += list(np.array(new_pose3d))
print(cam_3ds[data_type][0])
    
data_total = {'train': {}, 'test': {}}

for data_type in ['train', 'test']:
    data_total[data_type]['joint_2d']          = copy.deepcopy(np.array(joint_2ds[data_type]))
    data_total[data_type]['confidence']        = copy.deepcopy(np.array(confidences[data_type]))
    data_total[data_type]['camera_name']       = copy.deepcopy(np.array(camera_names[data_type]))
    data_total[data_type]['action']            = copy.deepcopy(np.array(actions[data_type]))
    data_total[data_type]['source']            = copy.deepcopy(np.array(sources[data_type]))
    data_total[data_type]['camera_name']       = copy.deepcopy(np.array(camera_names[data_type]))
    data_total[data_type]['frame']             = copy.deepcopy(np.array(frames[data_type]))
    data_total[data_type]['cam_3d']            = copy.deepcopy(np.array(cam_3ds[data_type]))


if (len(data_total['train']['joint_2d']) == 0) or only_test:
    for key in data_total['train'].keys():
        data_total['train'][key] = copy.deepcopy(data_total['test'][key][:243])
        print(key, data_total['train'][key].shape)
data_total['train']['joint_2d'].shape, data_total['test']['joint_2d'].shape  

(2929, 16, 2) (2929, 16, 3)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.18546074e-01  3.13477341e-04 -2.11239503e-02]
 [-1.32954629e-01  4.72305861e-01  9.51011342e-02]
 [-1.53749059e-01  7.97275637e-01  2.73700839e-01]
 [ 1.18546074e-01 -3.13477341e-04  2.11239503e-02]
 [ 9.63895564e-02  4.76514175e-01  1.18332277e-01]
 [ 8.19574585e-02  8.15433042e-01  2.69529812e-01]
 [ 3.51583778e-03 -2.25936963e-01 -2.37523839e-02]
 [ 1.09323380e-02 -4.77327568e-01 -6.82279314e-02]
 [ 1.94244175e-02 -5.97434048e-01 -1.15704790e-01]
 [ 2.79164970e-02 -7.17540529e-01 -1.63181649e-01]
 [ 1.48406677e-01 -4.23220425e-01 -4.78323932e-02]
 [ 4.45392422e-01 -4.23356569e-01 -4.76242754e-02]
 [ 6.67393346e-01 -4.49128482e-01 -7.26417617e-02]
 [-1.21724075e-01 -4.25580827e-01 -6.81152010e-02]
 [-4.17845264e-01 -4.46781087e-01 -6.01286828e-02]
 [-6.39455101e-01 -4.48413944e-01 -9.83443022e-02]]
joint_2d (243, 17, 2)
confidence (243, 17, 1)
camera_name (243,)
action (243,)
source (243,)
frame (243,

((243, 17, 2), (2929, 17, 2))

In [25]:
data_total['test']['joint_2d'].shape, data_total['test']['confidence'].shape, data_total['test']['cam_3d'].shape

((2929, 17, 2), (2929, 17, 1), (2929, 17, 3))

In [26]:
save_path = os.path.join('/home/hrai/codes/MotionBERT', f'data/motion3d/poseaug_3dhp_test.pkl')
savepkl(data_total, save_path)